In [90]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn import preprocessing

from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import re

import warnings
warnings.filterwarnings('ignore')

In [91]:
df2 = pd.DataFrame(columns=['text', 'label'])
df = pd.read_csv('/Users/mac/Downloads/rusentitweet_train.csv')
for i, row in df.iterrows():
    if row[1] in ("negative", "positive"):
        df2.loc[len(df2.index)] = row
df = df2
df

text     label
0     Помойму я вкрашилась в Чимина🤧 https://t.co/t2...  positive
1                         @buybread_ я не с порядке!!!!  negative
2     @ange1flyhigh В следующий раз буду до победног...  positive
3     @LimitaVIP Удивительный гiмн...\r\nУдивительно...  negative
4                               я срала на эту биологию  negative
...                                                 ...       ...
4564  Дедрадио5 то есть ты хочешь сказать, что ты ла...  negative
4565  @kmoo_m ДЕЙСТВИТЕЛЬНО\r\nесли любовь, то тольк...  positive
4566  С Хоби утро может быть только добрым😂 https://...  positive
4567  Не успела встать, а уже заебалась, впрочем нич...  negative
4568  ах твиттер ивана урганта, а ведь всё так хорош...  negative

[4569 rows x 2 columns]

In [92]:
import nltk
from nltk.corpus import stopwords
 
nltk.download('stopwords')
stop = stopwords.words('russian')
stop.remove('не')
    
stop

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и',
 'в',
 'во',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'над',
 'б

In [93]:

s=[]
s1 = ""
s2 = ""
regs = [r'https?:\/\/.\S+', r'#\S+', r'@\S+', r'\\r\\n', r'[-.)(,:]']
glasn = ['о', 'о', 'а', 'я', 'а', 'а', 'и', 'а', 'е', 'у', 'о', 'о', 'а', 'е', 'я', 'о', 'у']
signs = ['!', '_', '.']

for i, row in df.iterrows():
    row[0] = row[0].lower()

df1 = df    

for i, row in df1.iterrows():
    for reg in regs:
        row[0] = re.sub(reg, "", row[0])
        
    # delete stop words
    for t in row[0].split():
        if t not in stop:
            s.append(t)
    row[0] = ' '.join(s)
    
    # delete repeats symbols not letters
    r = row[0]
    for t in range(0, len(row[0])):
        if not(r[t].isalpha()) and r[t] == r[t-1]:
            continue
        else:
            s1+=r[t]
    row[0] = s1
    
    s1 = ""
    
    # удаление подряд идущих гласных
    r = row[0]
    for t in range(0, len(row[0])):
        if (r[t] in glasn) and r[t] == r[t-1]:
            continue
        else:
            s1+=r[t]
    row[0] = s1
    
    s1 = ""
    r = row[0]
    for t in range(0, len(row[0])):
        if (r[t].isdigit()):
            continue
        else:
            s1+=r[t]
    row[0] = s1
    
    r = row[0] + " "
    
    #отделение смайлика от слова 
    for t in range(0, len(row[0])):
        if not(r[t+1].isalpha()) and r[t]!= " " and not(r[t+1].isdigit()) and not(r[t+1] in signs):
            s2+=r[t] + " "
        else:
            s2+=r[t]
    row[0] = s2
    
    s = []
    s2=""
    s1=""
df1

text     label
0                        помойму  вкрашилась  чимина 🤧   positive
1                                         не  порядке!   negative
2     следующий  буду  победного  закрывать  пожелай...  positive
3     удивительный  гiмн  удивительно  пока  ещё  не...  negative
4                                      срала  биологию   negative
...                                                 ...       ...
4564  дедрадио  хочешь  сказать  лайкаешь  не  мои  ...  negative
4565                      действительно  любовь  такая   positive
4566                              хоби  утро  добрым 😂   positive
4567             не  успела  встать  заебалась  нового   negative
4568  ах  твиттер  ивана  урганта  всё  начиналось  ...  negative

[4569 rows x 2 columns]

In [94]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from functools import reduce
nltk.download('punkt')

new = SnowballStemmer(language='russian')
s=[]
for i, row in df1.iterrows():

    for t in row[0].split():
        s.append(new.stem(t))
    row[0] = ' '.join(s)
    s=[]
df

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


text     label
0                                  помойм вкраш чимин 🤧  positive
1                                           не порядке!  negative
2                 след буд победн закрыва пожела удачи!  positive
3     удивительн гимн удивительн пок ещ не сдохл украин  negative
4                                           срал биолог  negative
...                                                 ...       ...
4564  дедрад хочеш сказа лайка не мо твит ? 😡 пздц у...  negative
4565                             действительн любов так  positive
4566                                     хоб утр добр 😂  positive
4567                           не успел встат заеба нов  negative
4568        ах твиттер ива ургант все начина все потеря  negative

[4569 rows x 2 columns]

In [95]:
arr = []

for i in range(0, len(df1)):
    arr.append([])

for i, row in df1.iterrows():
    for t in row[0].split():
        arr[i].append(t)
# arr
b = []
for i, row in df1.iterrows():
    b.append(row[0])
b

['помойм вкраш чимин 🤧',
 'не порядке!',
 'след буд победн закрыва пожела удачи!',
 'удивительн гимн удивительн пок ещ не сдохл украин',
 'срал биолог',
 'помим алин ещ радост гемоглобин повыс',
 'пиздец че вобщ чувств жизн помота прошл сутк туман',
 'спичк ? зажеч огон тво глаз',
 'эт сам дел очен крут',
 'хоч сказа чтот приятн получа "ид нах ебла противн "',
 'сук огреба',
 'темн прошл патриарх кирилл владык стал миллиардер — инсайд',
 'проигра укулел кукушк ком интересн очен прост аккорд темп вполн легк крут',
 'лезбиянк эт твит 🤔',
 'классспасиб доставк galaxy buds bts украинупоиск папик отмен',
 'бля паладин неубива',
 'отц сдела ментальн инвалид',
 'эт точно! 👍',
 'пок имен любл',
 'й красив числ',
 'заплат сем минут потрат узна основн тип рифмовк',
 'соболезн',
 'закр вскро',
 'нрав професс содержанк посовет вуз сответств программ бакалавриат / магистратур',
 'пиздец какойт поход осозна сон слов',
 'больш сем не крыс',
 'разыгрыва 🤣 написа валентинк препод физик подлож лиц неопи

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


vec = TfidfVectorizer()
matr = vec.fit_transform(b)
# tf-idf
print(matr)
print(vec.get_feature_names_out())

vect = CountVectorizer()
m = vect.fit_transform(b)
fm = vect.get_feature_names_out()
# count matrix
print(m.toarray())


  (0, 7680)	0.5346549574878128
  (0, 831)	0.5831906314872191
  (0, 5064)	0.6115821807240984
  (1, 5148)	0.9574160277151054
  (1, 3723)	0.28871188038289775
  (2, 7108)	0.4569916101707917
  (2, 4928)	0.4357766365990583
  (2, 2068)	0.42072437330419066
  (2, 4768)	0.4569916101707917
  (2, 609)	0.2764794202338423
  (2, 6256)	0.37278169396629607
  (3, 7153)	0.3265203201691811
  (3, 6092)	0.34993117747630964
  (3, 1874)	0.22737641605256803
  (3, 4968)	0.262987212267135
  (3, 1229)	0.38169773142733265
  (3, 7114)	0.6998623549526193
  (3, 3723)	0.12070571277744499
  (4, 459)	0.6843936599995228
  (4, 6534)	0.7291126923545203
  (5, 4796)	0.4538197839538238
  (5, 1214)	0.4538197839538238
  (5, 5703)	0.3756688108279149
  (5, 197)	0.41780426587187935
  (5, 5051)	0.4538197839538238
  :	:
  (4564, 4628)	0.3615276125809485
  (4564, 7545)	0.336366909824546
  (4564, 2983)	0.37446199232746924
  (4564, 3406)	0.20570579203467215
  (4564, 6792)	0.2666510057531861
  (4564, 6198)	0.251718044820868
  (4564, 372